In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from transformator.tree import Tree
from transformator.lambda_tree import LambdaTree
import pyAgrum.lib.notebook as gnb

In [4]:
N = 6

In [5]:
ns = list(range(N-4, N+1))

In [6]:
classes = [list(LambdaTree.generate(k)) for k in ns]

In [7]:
[len(c) for c in classes]

[2, 4, 9, 22, 57]

In [8]:
# n-4:
# l^2*s^2*n*B(n)
#
# n-3:
# ((4*a*o*s*u - 2*l^2*s - 2*l*s^2)*(n - 1) + 6*a*o*s*u - 2*l^2*s - 3*l*s^2)*B(n)
# ((4*n+2)*a*o*s*u - 2*n*l^2*s - (2*n+1)*l*s^2)*B(n)
#
# n-2:
# ((-4*a*o*u + l^2 + 4*l*s + s^2)*(n - 2) - 8*u*o*a + 2*l^2 + 10*l*s + 3*s^2)*B(n)
# ((-4*n)*a*o*u + l^2*n + (4*n+2)*l*s + (n+1)*s^2)*B(n)
#
# n-1:
# ((-2*l - 2*s)*(n - 3) - 7*l - 8*s)*B(n)
# ((-2*n - 1)*l + (-2*n-2)*s)*B(n)
#
# n:
# (1 + n)*B(n)

r_ns = [
    ns[0] * len(classes[0]),
    ((4*ns[1]+2) - (2*ns[1]) - (2*ns[1]+1)) * len(classes[1]),
    (-4*ns[2] + ns[2] + (4*ns[2]+2) + (ns[2]+1)) * len(classes[2]),
    ((-2*ns[3]-1) + (-2*ns[3]-2)) * len(classes[3]),
    (ns[4] + 1) * len(classes[4]),
]

r_ns

[4, 4, 99, -506, 399]

In [9]:
sum(r_ns)

0

In [10]:
r_classes = [[] for _ in range(5)]

In [38]:
def clear_r_class(k):
    r_classes[k] = []

def append_r_class_subtree_visitor(k, multiplier, visitor):
    trees = sum(
        [
            list(t.visit_subtrees(visitor))
            for t
            in classes[i]
        ],
        []
    )

    r_classes[k].append((multiplier, trees))

def print_r_class_stats(k):
    total = 0

    for _, trees in r_classes[k]:
        gnb.sideBySide(*[t for t in trees[:5]])

    for multiplier, trees in r_classes[k]:
        print(f"{multiplier} x {len(trees)}")
        total += multiplier * len(trees)

    print(f"total: {total}")
    print(f"expected: {r_ns[k]}")

In [48]:
# n:
# (1 + n)*B(n)

i = 4

clear_r_class(i)

# 1 + n

append_r_class_subtree_visitor(i, 1,
    lambda kind, left, right:
        None if kind[0] is not None else
        [kind + ("*", )] + left + right
)

print_r_class_stats(i)


1 x 399
total: 399
expected: 399


In [105]:
# n-1:
# ((-2*n - 1)*l + (-2*n-2)*s)*B(n)

i = 3

clear_r_class(i)

# (-2*n - 1)*l

append_r_class_subtree_visitor(i, -1,
    lambda kind, left, right:
        ["l", kind] + left + right + [(None, "*")]
)

# (-2*n-2)*s

append_r_class_subtree_visitor(i, -1,
    lambda kind, left, right:
        None if kind[0] is not None else
        ["s", kind, (None, "*")]
)

append_r_class_subtree_visitor(i, -1,
    lambda kind, left, right:
        None if kind[0] is not None else
        ["s", (None, "*"), kind]
)

print_r_class_stats(i)

-1 x 242
-1 x 132
-1 x 132
total: -506
expected: -506


In [54]:
# n-2:
# ((-4*n)*a*o*u + l^2*n + (4*n+2)*l*s + (n+1)*s^2)*B(n)

i = 2

clear_r_class(i)

# (-4*n)*a*o*u

append_r_class_subtree_visitor(i, -1,
    lambda kind, left, right:
        None if kind[0] is None else
        ["a", "o", (None, "*"), None, kind] + left + right
)

append_r_class_subtree_visitor(i, -1,
    lambda kind, left, right:
        None if kind[0] is None else
        ["a", "o", None, (None, "*"), kind] + left + right
)

append_r_class_subtree_visitor(i, -1,
    lambda kind, left, right:
        None if kind[0] is None else
        ["a", kind] + left + right + ["o", (None, "*"), None]
)

append_r_class_subtree_visitor(i, -1,
    lambda kind, left, right:
        None if kind[0] is None else
        ["a", kind] + left + right + ["o", None, (None, "*")]
)

# l^2*n

append_r_class_subtree_visitor(i, 1,
    lambda kind, left, right:
        None if kind[0] is None else
        ["l", "l", kind] + left + right + [None, None]
)

# (4*n+2)*l*s

append_r_class_subtree_visitor(i, 1,
    lambda kind, left, right:
        ["l", "s", (None, "*"), kind] + left + right + [None]
)

append_r_class_subtree_visitor(i, 1,
    lambda kind, left, right:
        ["l", "s", None, kind] + left + right + [(None, "*")]
)

# (n+1)*s^2

append_r_class_subtree_visitor(i, 1,
    lambda kind, left, right:
        None if kind[0] is not None else
        ["s", None, "s", None, kind] + left + right
)

print_r_class_stats(i)

-1 x 36
-1 x 36
-1 x 36
-1 x 36
1 x 36
1 x 81
1 x 81
1 x 45
total: 99
expected: 99


In [56]:
# n-3:
# ((4*n+2)*a*o*s*u - 2*n*l^2*s - (2*n+1)*l*s^2)*B(n)

i = 1

clear_r_class(i)

# (4*n+2)*a*o*s*u

append_r_class_subtree_visitor(i, 1,
    lambda kind, left, right:
        ["a", "s", None, "o", (None, "*"), None, kind] + left + right
)

append_r_class_subtree_visitor(i, 1,
    lambda kind, left, right:
        ["a", "s", None, "o", None, (None, "*"), kind] + left + right
)

# - 2*n*l^2*s

append_r_class_subtree_visitor(i, -1,
    lambda kind, left, right:
        None if kind[0] is None else
        ["l", "l", "s", None, kind] + left + right + [(None, "*"), None]
)

append_r_class_subtree_visitor(i, -1,
    lambda kind, left, right:
        None if kind[0] is None else
        ["l", "l", "s", None, kind] + left + right + [None, (None, "*")]
)

# - (2*n+1)*l*s^2

append_r_class_subtree_visitor(i, -1,
    lambda kind, left, right:
        ["l", "s", None, "s", None, kind] + left + right + [None]
)

print_r_class_stats(i)

1 x 28
1 x 28
-1 x 12
-1 x 12
-1 x 28
total: 4
expected: 4


In [106]:
# n-4:
# l^2*s^2*n*B(n)

i = 0

clear_r_class(i)

# l^2*s^2*n

append_r_class_subtree_visitor(i, 1,
        lambda kind, left, right:
            None if kind[0] is None else
            ["l", "l", "s", None, "s", None, kind] + left + right + [None, None]
)

print_r_class_stats(i)

1 x 4
total: 4
expected: 4


In [107]:
minus = []
plus = []

for r_class_id, r_class in enumerate(r_classes):
    for r_subclass_id, (multiplier, r_subclass) in enumerate(r_class):
        for tree in r_subclass:
            data = (r_class_id, r_subclass_id, tree)

            if multiplier == -1:
                minus.append(data)
            elif multiplier == 1:
                plus.append(data)
            else:
                assert False

minus.sort(key=lambda data: str(data[2]))
plus.sort(key=lambda data: str(data[2]))

print(f"minus: {len(minus)}")
print(f"plus: {len(plus)}")

minus: 702
plus: 702


In [119]:
minus[0]

(3, 0, <@, @, l, o, , , *, o, , , o, , >)

In [120]:
plus[0]

(4, 0, <@, @, l, o, *, , , o, , , o, , >)